In [9]:
import psycopg2
from psycopg2.extensions import Column
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Get database URLs from environment variables
source_db_url = os.getenv("DB_URL")
backup_db_url = os.getenv("NEW_DB_URL")

# Table name for the copy operation
table_name = "usd_er"

try:
    # Connect to the source and backup databases
    conn_source = psycopg2.connect(source_db_url)
    conn_backup = psycopg2.connect(backup_db_url)

    # Create cursors for both databases
    cursor_source = conn_source.cursor()
    cursor_backup = conn_backup.cursor()

    # Fetch data from the source table (just the structure, not data)
    cursor_source.execute(f"SELECT * FROM {table_name} WHERE 1=0;")
    data_to_copy = cursor_source.fetchall()

    # Drop the backup table if it exists
    cursor_backup.execute(f"DROP TABLE IF EXISTS {table_name};")

    # Create the backup table
    create_table_query = f"CREATE TABLE {table_name} ("
    create_table_query += ", ".join([f"{col[0]} {('timestamp' if col[1] == 1114 else 'float8')}" for col in cursor_source.description])
    create_table_query += ");"

    cursor_backup.execute(create_table_query)

    # Insert data into the backup table
    for row in data_to_copy:
        placeholders = ', '.join(['%s'] * len(row))
        insert_query = f"INSERT INTO {table_name} VALUES ({placeholders});"
        cursor_backup.execute(insert_query, row)

    # Commit changes to the backup database
    conn_backup.commit()

    # Close the cursors and connections
    cursor_source.close()
    cursor_backup.close()
    conn_source.close()
    conn_backup.close()

    print(f"The table {table_name} has been successfully copied to the backup database.")
except Exception as e:
    print("An error occurred:", e)

The table usd_er has been successfully copied to the backup database.


In [12]:
print(data_to_copy)


[]
